In [1]:
import os
import shutil
import random
import cv2 as cv
import numpy as np

In [2]:
def split_data(source_dir, train_dir, test_dir, test_size=0.2):
    try:
        # Sikre, at destinationsmapperne eksisterer
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Hent alle filnavne fra data-mappen
        all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

        # Bland listen og lav et tilfældigt split
        random.shuffle(all_files)
        split_point = int((1 - test_size) * len(all_files))

        # Fordel filerne i train og test mapper
        train_files = all_files[:split_point]
        test_files = all_files[split_point:]

        # Funktion til at kopiere filer til den angivne mappe
        def copy_files(files, destination):
            for f in files:
                shutil.copy(os.path.join(source_dir, f), os.path.join(destination, f))

        # Kopier filerne
        copy_files(train_files, train_dir)
        copy_files(test_files, test_dir)

        print(f'{len(train_files)} filer blev kopieret til træningsmappen.')
        print(f'{len(test_files)} filer blev kopieret til testmappen.')
    except FileNotFoundError as fnf_error:
        print(f"Fejl: {fnf_error}")
    except Exception as e:
        print(f"En uventet fejl opstod: {e}")

In [3]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

def zoom_tile(tile, crop_percentage=0.10):
    height, width = tile.shape[:2]
    
    # Beregner hvor meget der skal beskæres fra hver side
    crop_height = int(height * crop_percentage)
    crop_width = int(width * crop_percentage)
    
    # Opdaterer udsnittets start- og slutpunkter for at fjerne de yderste 5%
    cropped_tile = tile[crop_height:height-crop_height, crop_width:width-crop_width]
    
    return cropped_tile

def remove_circle_from_tile(tile):
    # Konverterer til RGBA hvis billedet ikke allerede er det
    if tile.shape[2] == 3:
        tile = cv.cvtColor(tile, cv.COLOR_BGR2BGRA)
    
    height, width = tile.shape[:2]
    # Finder diameteren for cirklen som 5% af den mindste dimension af billedet
    diameter = int(0.70 * min(width, height))
    
    # Beregner centrum for cirklen
    center = (width // 2, height // 2)
    
    # Tegner cirklen i alfakanalen
    mask = np.zeros((height, width), dtype=np.uint8)
    cv.circle(mask, center, diameter // 2, (255), -1)
    
    # Fjerner cirklen fra alfakanalen, så den bliver gennemsigtig
    tile[mask == 255, 3] = 0

    return tile

def save_tile(tile, output_folder, image_name, x, y):
    # her laver vi en mappe 'blandet' hvis den ikke findes
    if not os.path.exists(os.path.join(output_folder, "blandet")):
        os.makedirs(os.path.join(output_folder, "blandet"))

    # her definerer vi navnet på det tile der skal gemmes som f.eks. 1_3_2.png ved brug af en f-streng
    tile_filename = f"{image_name}_{x}_{y}.png"

    # her definerer bi tile_path som er stedet hvor vi vil gemme vores tile
    tile_path = os.path.join(output_folder, "blandet", tile_filename)
    # her gemmer vi vores tile som tile_filename i folderen 'blandet'
    cv.imwrite(tile_path, tile)

    # her skriver vi til konsollen at vi har gemt vores til i 'blandet' folderen
    print(f"Saved Tile as {tile_filename} in 'blandet' folder")

In [4]:
def tile_set_create(output_folder, input_folder):
    # foldere til billeder defineres
    output_folder = "Data/KD train tiles"
    input_folder = "Data/KD train plader"

    # vi kigger igennem hver fil i input folderen
    for filename in os.listdir(input_folder):
        
        if not filename.endswith('.jpg'):
            continue
        # da billederne er navngivet som 1.jpg, 2.jpg osv. 
        # bruger vi filename counteren til at definere billedet vi vil have
        image_path = input_folder +"/" +filename
        # vi udskriver stien til billedet som man kan kigge på hvis der skulle opstå fejl
        print(image_path)
        # her tjekker vi om billedet findes i mappen og skriver "Image not found" hvis det ikke eksisterer
        if not os.path.isfile(image_path):
            print("Image not found")
            return
        
        # her bruger vi openCV til at åbne billedet
        image = cv.imread(image_path)
        tiles = get_tiles(image)
        print(len(tiles))
        for y, row in enumerate(tiles):
            for x, tile in enumerate(row):
                tile = zoom_tile(tile)
                #tile = remove_circle_from_tile(tile)
                save_tile(tile, output_folder, filename, x, y)



In [5]:
def main():
    tiles_folder = "Data/KD train tiles/blandet"
    csv_document = "Data/blandet.csv"

    images = load_images_from_folder(tiles_folder)
    with open(csv_document, "w", newline='') as file:
        writer = csv.writer(file)
        # Skriv header til CSV-filen
        writer.writerow(["Filename", "Hue", "Saturation", "Value"])
        for filename, tile in images:
            hsv_tile = cv.cvtColor(tile, cv.COLOR_BGR2HSV)
            hue, saturation, value = np.median(hsv_tile, axis=(0,1))
            print(filename, hue, saturation, value)
            write_to_csv(filename, hue, saturation, value, writer)

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder, filename))
        if img is not None and img.any():
            images.append((filename, img))
    return images

def write_to_csv(filename, hue, saturation, value, writer):
    writer.writerow([filename, hue, saturation, value])

In [6]:
# Stier til mapperne
data_dir = 'Data/KD hele plader'
train_dir = 'Data/KD train plader'
test_dir = 'Data/KD test plader'
tile_train_dir = "Data/KD train tiles"

In [7]:
split_data(data_dir, train_dir, test_dir)

58 filer blev kopieret til træningsmappen.
15 filer blev kopieret til testmappen.


In [8]:
tile_set_create(tile_train_dir, train_dir)

Data/KD train plader/63.jpg
5
Saved Tile as 63.jpg_0_0.png in 'blandet' folder
Saved Tile as 63.jpg_1_0.png in 'blandet' folder
Saved Tile as 63.jpg_2_0.png in 'blandet' folder
Saved Tile as 63.jpg_3_0.png in 'blandet' folder
Saved Tile as 63.jpg_4_0.png in 'blandet' folder
Saved Tile as 63.jpg_0_1.png in 'blandet' folder
Saved Tile as 63.jpg_1_1.png in 'blandet' folder
Saved Tile as 63.jpg_2_1.png in 'blandet' folder
Saved Tile as 63.jpg_3_1.png in 'blandet' folder
Saved Tile as 63.jpg_4_1.png in 'blandet' folder
Saved Tile as 63.jpg_0_2.png in 'blandet' folder
Saved Tile as 63.jpg_1_2.png in 'blandet' folder
Saved Tile as 63.jpg_2_2.png in 'blandet' folder
Saved Tile as 63.jpg_3_2.png in 'blandet' folder
Saved Tile as 63.jpg_4_2.png in 'blandet' folder
Saved Tile as 63.jpg_0_3.png in 'blandet' folder
Saved Tile as 63.jpg_1_3.png in 'blandet' folder
Saved Tile as 63.jpg_2_3.png in 'blandet' folder
Saved Tile as 63.jpg_3_3.png in 'blandet' folder
Saved Tile as 63.jpg_4_3.png in 'blande